<a href="https://colab.research.google.com/github/Echo271/Naive-Bayes-Classifier/blob/main/NB_Dataset_Indo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

sms = pd.read_csv('/content/Data Training - Sheet 1 (1).csv', sep=',', engine='python')
sms = sms[['Teks', 'label']]
sms.columns = ['pesan', 'label']
sms.tail()

,pesan,label
794,"wah repot kalo asumsinya gitu, kadang ada yg g...",normal
795,Slmt!! No Anda Men-dptkan Hadiah dr Pengundian...,penipuan
796,Lingkungan kerjanya berasap. Kalo gak ngerokok...,normal
797,"GRATIS Internetan hg 1000 MB, Nelpon 1000 Meni...",penipuan
798,"RAWIT Kuotanya Bagus, Harganya Khusus. AYO isi...",promo


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = stopwords.words("indonesian")
print(len(stop_words))
print(stop_words[:10])

758
['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir']


In [ ]:
def remove_stopwords(text):
    """Function to replace stopords with an empty space and removes double spaces
    removing all chars other then alphabet and stemming words
    
    returns: corpus of stemmed words"""

    text = re.sub('^a-zA-Z',' ',text)
    text = text.split()
    text = [ps.stem(word) for word in text if word not in stop_words]
    text = ' '.join(text).replace('  ', ' ')
    return text

In [ ]:
#Filtering
sms['teks_baru']= sms.pesan.apply(remove_stopwords)
sms.tail()

,pesan,label,teks_baru
794,"wah repot kalo asumsinya gitu, kadang ada yg g...",normal,"repot kalo asumsinya gitu, kadang yg ganti kel..."
795,Slmt!! No Anda Men-dptkan Hadiah dr Pengundian...,penipuan,slmt!! No anda men-dptkan hadiah dr pengundian...
796,Lingkungan kerjanya berasap. Kalo gak ngerokok...,normal,lingkungan kerjanya berasap. kalo gak ngerokok...
797,"GRATIS Internetan hg 1000 MB, Nelpon 1000 Meni...",penipuan,"grati internetan hg 1000 mb, nelpon 1000 menit..."
798,"RAWIT Kuotanya Bagus, Harganya Khusus. AYO isi...",promo,"rawit kuotanya bagus, harganya khusus. ayo isi..."


Stemming dan Tokenizing dengan matrix

In [ ]:
# Vectorizing
from sklearn.feature_extraction.text import CountVectorizer
pesan_promo = [row['pesan'] for index,row in sms.iterrows() if row['label']=='promo']
pesan_promo

In [ ]:
#vectorizing promo
vec_promo = CountVectorizer()
x_promo = vec_promo.fit_transform(pesan_promo)

tdm_promo = pd.DataFrame(x_promo.toarray(),columns=vec_promo.get_feature_names())
tdm_promo

In [ ]:
#vectorizing penipuan
pesan_penipuan = [row['pesan'] for index,row in sms.iterrows() if row['label']=='penipuan']
vec_penipuan = CountVectorizer()
x_penipuan = vec_penipuan.fit_transform(pesan_penipuan)

tdm_penipuan = pd.DataFrame(x_penipuan.toarray(),columns=vec_penipuan.get_feature_names())
tdm_penipuan

In [ ]:
#vectorizing umum
pesan_umum = [row['pesan'] for index,row in sms.iterrows() if row['label']=='normal']
vec_umum = CountVectorizer()
x_umum = vec_umum.fit_transform(pesan_umum)

tdm_umum = pd.DataFrame(x_umum.toarray(),columns=vec_umum.get_feature_names())
tdm_umum

Pesan Promo

In [ ]:
#Frequency term 
list_kata_promo = vec_promo.get_feature_names();
hitung_kata_promo = x_promo.toarray().sum(axis=0)
freq_promo = dict(zip(list_kata_promo,hitung_kata_promo))
list_kata_promo
freq_promo

In [ ]:
#Persentase term
prob_promo = []
for word,count in zip(list_kata_promo,hitung_kata_promo):
  prob_promo.append(count/len(list_kata_promo))
dict(zip(list_kata_promo,prob_promo))

Pesan Penipuan

In [ ]:
list_kata_penipuan = vec_penipuan.get_feature_names();
hitung_kata_penipuan = x_penipuan.toarray().sum(axis=0)
freq_penipuan = dict(zip(list_kata_penipuan,hitung_kata_penipuan))
list_kata_penipuan
freq_penipuan

In [ ]:
prob_penipuan = []
for word,count in zip(list_kata_penipuan,hitung_kata_penipuan):
  prob_penipuan.append(count/len(list_kata_penipuan))
dict(zip(list_kata_penipuan,prob_penipuan))

Pesan Umum

In [ ]:
list_kata_umum = vec_umum.get_feature_names();
hitung_kata_umum = x_umum.toarray().sum(axis=0)
freq_umum = dict(zip(list_kata_umum,hitung_kata_umum))
list_kata_umum
freq_umum

In [ ]:
prob_umum = []
for word,count in zip(list_kata_umum,hitung_kata_umum):
  prob_umum.append(count/len(list_kata_umum))
dict(zip(list_kata_umum,prob_umum))

Total keseluruhan kata sebagai pembanding

In [ ]:
text = [row['pesan'] for index,row in sms.iterrows()]
vec = CountVectorizer()
x = vec.fit_transform(text)

total_features = len(vec.get_feature_names())
total_features

3909

In [ ]:
total_cnts_features_promo = hitung_kata_promo.sum(axis=0)
total_cnts_features_penipuan = hitung_kata_penipuan.sum(axis=0)
total_cnts_features_umum = hitung_kata_umum.sum(axis=0)
print(total_cnts_features_promo)
print(total_cnts_features_penipuan)
print(total_cnts_features_umum)

3882
5533
4742


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Upload File Uji

In [ ]:
datatest = pd.read_csv('/content/Data uji - Sheet1 (1).csv', sep=',', engine='python')
datatest = datatest[['Teks','label']]
datatest.columns = ['pesan','label']
datatest.tail()

,pesan,label
337,5 HARI LAGI ! EKSTRA Pulsa 50rb dg beli paket ...,promo
338,Mau cepat dan ga mau ribet cek pulsa & sisa ku...,promo
339,MAU BONUS PULSA 50% dari Indosat Ooredoo?? Ket...,promo
340,SELAMAT Anda sebagai pemenang promo UNDIAN POP...,penipuan
341,Ayo dukung pelestarian alam Indonesia bersama ...,promo


In [ ]:
from nltk.tokenize import word_tokenize

def pesan(text):
  new_sentence = text
  new_word_list = word_tokenize(new_sentence) #tokenize data uji
  return new_word_list

def pred_promo(wd): 
  list_kata_baru = wd
  prob_promo_with_ls = []
  for word in list_kata_baru:
    if word in freq_promo.keys():
      count = freq_promo[word]
    else:
      count = 0
    prob_promo_with_ls.append((count + 1)/(total_cnts_features_promo + total_features))
  return prob_promo_with_ls

def pred_penipuan(wd):
  list_kata_baru = wd
  prob_penipuan_with_ls = []
  for word in list_kata_baru:
    if word in freq_penipuan.keys():
      count = freq_penipuan[word]
    else:
      count = 0
    prob_penipuan_with_ls.append((count + 1)/(total_cnts_features_penipuan + total_features))
  return prob_penipuan_with_ls

def pred_umum(wd):
  list_kata_baru = wd
  prob_umum_with_ls = []
  for word in list_kata_baru:
    if word in freq_umum.keys():
      count = freq_umum[word]
    else:
      count = 0
    prob_umum_with_ls.append((count + 1)/(total_cnts_features_umum + total_features))
  return prob_umum_with_ls

def nilai_promo(n):
  prob_promo_with_ls = n
  i = 0
  prob_pesan_promo = 1
  for i in range(len(prob_promo_with_ls)):
    prob_pesan_promo = prob_pesan_promo*prob_promo_with_ls[i]
  return prob_pesan_promo

def nilai_penipuan(n):
  prob_penipuan_with_ls = n
  i = 0
  prob_pesan_penipuan = 1
  for i in range(len(prob_penipuan_with_ls)):
    prob_pesan_penipuan = prob_pesan_penipuan*prob_penipuan_with_ls[i]
  return prob_pesan_penipuan

def nilai_umum(n):
  prob_umum_with_ls = n
  i = 0
  prob_pesan_umum = 1
  for i in range(len(prob_umum_with_ls)):
    prob_pesan_umum = prob_pesan_umum*prob_umum_with_ls[i]
  return prob_pesan_umum

def hasil(text,pr,pn,um):
  new_sentence = text
  prob_pesan_promo = pr
  prob_pesan_penipuan = pn
  prob_pesan_umum = um

  promo = prob_pesan_promo*0.5
  penipuan = prob_pesan_penipuan*0.5
  umum = prob_pesan_umum*0.5

  if (promo > penipuan and promo > umum):
    prediksi = 'promo'
  elif (penipuan > promo and penipuan > umum):
    prediksi = 'penipuan'
  else:
    prediksi = 'normal'
  return prediksi

In [ ]:
def runNb(text):
 pesan(text)
 pred_promo(pesan(text))
 pred_penipuan(pesan(text))
 pred_umum(pesan(text))
 nilai_promo(pred_promo(pesan(text)))
 nilai_penipuan(pred_penipuan(pesan(text)))
 nilai_umum(pred_umum(pesan(text)))
 hasil(text,nilai_promo(pred_promo(pesan(text))),nilai_penipuan(pred_penipuan(pesan(text))),nilai_umum(pred_umum(pesan(text))))
 return hasil(text,nilai_promo(pred_promo(pesan(text))),nilai_penipuan(pred_penipuan(pesan(text))),nilai_umum(pred_umum(pesan(text))))

In [ ]:
datatest['prediksi'] = datatest.pesan.apply(runNb)
datatest

,pesan,label,prediksi
0,Saya blm dpt ttd nama1. Paling kalian duluan a...,normal,normal
1,MURAH! Hanya Rp25rb dptkan 2GB paket interneta...,promo,promo
2,Tersrrah aja. Pagi juga nggak pa2,normal,normal
3,Pilihannya hemat 2 jt atau lulus oct kan nanggung,normal,normal
4,"ini nomor baru papa, tolong kirimin pulsa 50.0...",penipuan,normal
...,...,...,...
337,5 HARI LAGI ! EKSTRA Pulsa 50rb dg beli paket ...,promo,promo
338,Mau cepat dan ga mau ribet cek pulsa & sisa ku...,promo,promo
339,MAU BONUS PULSA 50% dari Indosat Ooredoo?? Ket...,promo,promo
340,SELAMAT Anda sebagai pemenang promo UNDIAN POP...,penipuan,penipuan


In [ ]:
betul = 0
salah = 0
total = 0
for i in range(len(datatest)):
  if (datatest['label'][i] == datatest['prediksi'][i]):
    betul += 1
  else:
    salah += 1
  total += 1

print("Benar = ", betul)
print("salah = ", salah)
print("Tingkat akurasi = ", betul/total*100,"%")

Benar =  269
salah =  73
Tingkat akurasi =  78.65497076023392 %


In [ ]:
hasil_pred = datatest.to_csv('Data hasil prediksi.csv', index=False)

from google.colab import files
files.download("Data hasil prediksi.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>